In [132]:
import pandas as pd
import numpy as np

In [133]:
sales=pd.read_csv('kc_house_data_small.csv')

In [134]:
train=pd.read_csv('kc_house_data_small_train.csv')
test=pd.read_csv('kc_house_data_small_test.csv')
validate=pd.read_csv('kc_house_data_validation.csv')

In [135]:
def get_numpy_data(dataframe,features,output):
    dataframe['constant']=1
    features=['constant']+features
    feature_matrix=dataframe[features].values
    output_array=dataframe[output].values
    return feature_matrix,output_array

In [148]:
def normalize_features(features):
    norms=np.sqrt((features**2).sum(axis=0))
    norm_matrix=features/norms
    return norm_matrix,norms

In [149]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [150]:
all_feat=[m for m,n in dtype_dict.items() if n!=str]

In [151]:
all_feat.remove('price')
all_feat.sort()
len(all_feat)

17

In [152]:
feats_train,output_train=get_numpy_data(train,all_feat,'price')
feats_test,output_test=get_numpy_data(test,all_feat,'price')
feats_valid,output_valid=get_numpy_data(validate,all_feat,'price')

In [153]:
norm_train,norms=normalize_features(feats_train)

In [154]:
norm_test=feats_test/norms
norm_valid=feats_valid/norms

In [155]:
print (norm_test[0])
print (norm_train[9])

[ 0.01345102  0.01807473  0.01551285  0.0116321   0.017059    0.01564352
  0.01345387 -0.01346922  0.01362084  0.02481682  0.01759212  0.01375926
  0.00160518  0.0016225   0.05102365  0.          0.01350306  0.        ]
[ 0.01345102  0.00602491  0.01163464  0.01938684  0.01279425  0.01390535
  0.01346821 -0.01346251  0.0096309   0.          0.0083488   0.01195898
  0.00050756  0.00156612  0.          0.          0.01302544  0.        ]


In [156]:
def compute_1_distance(query,reference):
    sqr_sum=((query-reference)**2).sum(axis=0)
    distance=np.sqrt(sqr_sum)
    return distance

In [157]:
compute_distance(norm_test[0],norm_train[9])

0.05972359371398078

In [158]:
dis_list=[]
for i in range(10):
    dis_list.append(compute_distance(norm_test[0],norm_train[i]))

In [160]:
dis_list.index(min(dis_list))

8

In [162]:
diff=norm_train-norm_test[0]

In [163]:
diff[-1].sum()

-0.09343399874654643

In [164]:
np.sum(diff**2,axis=1)[15]

0.0033070590284564453

In [167]:
np.sum(diff[15]**2,axis=0)

0.0033070590284564453

In [168]:
def compute_mat_distance(query,reference):
    sqr_sum=((query-reference)**2).sum(axis=1)
    distance=np.sqrt(sqr_sum)
    return distance

In [169]:
d=compute_mat_distance(norm_test[0],norm_train)

In [170]:
d[100]

0.023708232416678195

In [171]:
d2=compute_mat_distance(norm_test[2],norm_train)

In [174]:
np.argmin(d2,axis=0)

382

In [175]:
output_train[382]

249000

In [182]:
def k_nearest_neighbors(k, feature_train, features_query):
    diff=feature_train-features_query
    distance=np.sum(diff**2,axis=1)
    sort_index=np.argsort(distance)
    neighbors=sort_index[0:k]
    return neighbors

In [212]:
a=k_nearest_neighbors(4,norm_train,norm_test[2])
output_train[a].sum()/4

413987.5

In [184]:
def predict(k, feature_train, output_train, features_query):
    diff=feature_train-features_query
    distance=np.sum(diff**2,axis=1)
    sort_index=np.argsort(distance)
    neighbors=sort_index[0:k]
    prediction=output_train[neighbors].mean()
    return prediction

In [185]:
predict(4,norm_train,output_train,norm_test[0])

923750.0

In [189]:
def predict_output(k, feature_train, output_train, features_query):
    a,b=features_query.shape
    predictions=[]
    for i in range(a):
        predictions.append(predict(k, feature_train, output_train, features_query[i]))
    return predictions

In [191]:
output_10=predict_output(10,norm_train,output_train,norm_test[0:9])

In [213]:
(min(output_10))

350032.0

In [206]:
rsslist=[]
for i in range(1,16):
    prep=predict_output(i,norm_train,output_train,norm_valid)
    rss=((prep-output_valid)**2).sum()
    rsslist.append((rss,i))
print(rsslist)

[(105453830251561.0, 1), (83445073504025.5, 2), (72692096019202.56, 3), (71946721652091.69, 4), (69846517419718.6, 5), (68899544353180.836, 6), (68341973450051.09, 7), (67361678735491.5, 8), (68372727958976.09, 9), (69335048668556.74, 10), (69523855215598.83, 11), (69049969587246.17, 12), (70011254508263.69, 13), (70908698869034.34, 14), (71106928385945.16, 15)]


In [202]:
rsslist.index(min(rsslist))

7

In [207]:
pre_test=predict_output(8,norm_train,output_train,norm_test)

In [208]:
rss_test=((pre_test-output_test)**2).sum()

In [209]:
rss_test

133118823551516.81